# Quickstart Guide for SuperAI 




## Setting Up the Environment


- SuperAI SDK (`pip install superai`)
-   You need to have valid AWS credentials in your environment.
- Make sure the `eu-central-1` AWS_REGION is set (`export AWS_DEFAULT_REGION=eu-central-1`)
-   You need to login to the SDK using
    `superai login -u <your-email-adress>`.
- If you run this file inside a jupyter-notebook process with AWS SSO, you should run it with credentials present. 
    E.g. `aws-vault exec {YOUR_PROFILE} -- jupyter-notebook ai_quickstart.ipynb`
- Optional: `! pip install ipywidgets`

In [ ]:
# Install widgets for progress bars/live updates
! pip install ipywidgets

### Model Class
First, its necessary to define the actual Python model.

To work with our platform, the model class needs to
be derived from `superai.meta_ai.BaseAI`.

Check out `./examples/ai/my_ai_project` for a minimal example.

In [ ]:
from pathlib import Path

PROJECT = Path("./examples/ai/my_ai_project")

In [ ]:
!ls {PROJECT}

In [ ]:
!cat {PROJECT}/"code"/MyAI.py

## Creating an AI object

An AI object is the main entity for working with SuperAI SDK. Its main
purpose is to store and manage the model source code, schema, and weights, and to
provide methods for building, deploying, and training the model.

To create an AI object, you'll need to specify the model class and the
path to the model. Here is an example:

In [ ]:
from superai.meta_ai.ai import AI

ai = AI(
    name="my-ai",
    model_class="MyAI",
    model_class_path=PROJECT/"code",
    weights_path=PROJECT/"weights",
)


The `ai` object is already useful for predicting locally for debugging.

In [ ]:
ai.predict(inputs={"values":"231231233"})

## Saving an AI Object to the Backend
To actually publish the AI for use in our platform you need to save the object to the database.
You can save it to the backend using the
`save` method.
Note that the name and version of the AI is unique for each user. Save will raise an exception if the AI already exists. In that case you can use `overwrite`.

In [ ]:
ai.save(overwrite=True) # Overwrite will allow you reusing the same name for subsequent calls

This will save the model to the backend and return the registered uuid.
In general, the AI object is responsible for storing the model source
code, a corresponding docker image for execution and default weights.

## Building the Model

Bulding a model means creating a docker image for the model. First it downloads a base image with some pre-packaged dependencies and then installs the model dependencies on top of it.


Issues:
- s2i not installed: You will see `ModuleNotFoundError: s2i is not installed`. Please install the package using 'brew install source-to-image' or read installation instructions at https://github.com/openshift/source-to-image#installation. You can install it using brew install source-to-image
- s2i image not found: You will see an error like this: Error: image docker.io/s2i-python36 not found. You can fix this by first running `aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 185169359328.dkr.ecr.us-east-1.amazonaws.com`and then running `docker pull 185169359328.dkr.ecr.us-east-1.amazonaws.com/superai-model-s2i-python310-cpu-internal-seldon:1`
- pulling image error: You will see and error like `pulling image error : Error response from   system_command.py:25 daemon: pull`. You can fix it by tagging the image `docker tag 185169359328.dkr.ecr.us-east-1.amazonaws.com/superai-model-s2i-python310-cpu-internal-seldon:1 superai-model-s2i-python310-cpu-internal-seldon:1`

You can build the model using the `build` method.
Building packages up your model (and its requirements) in a local docker image.

In [ ]:
ai.build()

## Pushing the Model to the Backend

You can push the AI docker image to the backend using the `push_image` method:

In [ ]:
ai.push_image()

## Creating an AI Instance

An AI itself is just a template for local development and for deriving
actual AI instances. AI instances are the main actors in the system.
They are the ones that are deployed, trained, and used for predictions.
AI instances are bound to a specific user ( #TODO and Organisations).

You can create an instance of your AI object using the `create_instance`
method:

In [ ]:
ai_instance = ai.create_instance()

# AI instances

## Deploying the Instance

You can deploy the instance using the `deploy` method:

In [ ]:
ai_instance.deploy(wait_time_seconds=30, redeploy=True) # Redeploy will automatically update existing deployments

In the backend, this will create a Deployment object, which is the
actual entity that is deployed to the cloud and used for predictions. By
default, deployments will automatically start and scale up when needed,
but it can also be configured to be online permanently. Note that each
ai instance has its own deployment, since each instance can have
different weights and configurations.

## Predicting with the AI Instance

You can make predictions with your AI instance using the `predict`
method:

In [ ]:
output = ai_instance.predict(input_data={"a":1}, wait_time_seconds=180)[0]

In [ ]:
print(output.prediction.__root__)
print(output.score)

This will return the prediction response. This operation can take some
time, since the deployment might need to be scaled up first. It can happen that it takes a few minutes (then you can increase the wait time).

# AI Checkpoints

Checkpoints are the objects representing weights for an AI.

If an AI instance is deployed we take the source code ( docker image) from the AI and combine it with the latest checkpoint chosen in the AI instance.


In [ ]:
current_checkpoint = ai_instance.get_checkpoint()
print(current_checkpoint)

A checkpoint can have a Tag (e.g. LATEST, STABLE, EXPERIMANTAL):

In [ ]:
current_checkpoint.tag

By default, the ai instance will always use the LATEST checkpoint.
When we train, a new Checkpoint will be created and it automatically gets the LATEST tag again.

In [ ]:
current_checkpoint.change_tag("STABLE")

In [ ]:
current_checkpoint.tag

In [ ]:
ai_instance.update(checkpoint_tag="STABLE")

## Training the AI Instance
Training an AI instance will create a new Checkpoint.
You can train an AI instance on an your local data or an app using the
`train_remote` method:

In [ ]:
training_instance = ai_instance.train_remote(local_path="<path-to-data>")

Replace `<path-to-data>` with the path to your training data.

or

In [ ]:
training_instance = ai_instance.train_remote(app_id="<app-id>")

Each training will create a new training instance object, which can be
used to monitor the training progress and results.

If the training completes, it will create a new checkpoint with the LATEST tag.

## Undeploying the Instance

Finally, when you're finished with the instance, you can undeploy it
using the `undeploy` method:

In [ ]:
ai_instance.undeploy()

By default, this is a non-blocking operation, meaning that the
deployment will be undeployed in the background. You can also wait for
the undeployment to finish by setting the `wait_seconds` parameter to
your desired timeout in seconds

In [ ]:
ai_instance.undeploy(wait_time_seconds=120)